In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as du
from torch.utils.data import IterableDataset

import numpy as np
import sys
import csv
from os import listdir
from tqdm import tqdm

In [2]:
class Price_Dataset(IterableDataset):
	def __init__(self, directory, window_size = 100, start = 0, end = 8):
		super().__init__()

		self.window_size = window_size

		prices = []
		files = listdir(directory)
		csvs = [f'{directory}/{f}' for f in files if f.endswith('csv')][start:end]
		for c in csvs:
			with open(c, 'r') as f:
				lines = list(csv.reader(f))[1:]
				prices.append(np.array([float(l[5]) for l in lines]))
		prices = np.array(prices)
		for i, row in enumerate(prices):
			for j, col in enumerate(row[:-1]):
				prices[i][j] -= prices[i][j+1]

		self.prices = prices
		self.length = int(len(self.prices)*(
			(len(self.prices[0]) - self.window_size-1)+(len(self.prices[0]) - self.window_size-1)//2+
			(len(self.prices[0]) - self.window_size-1)//3+(len(self.prices[0]) - self.window_size-1)//4+
			(len(self.prices[0]) - self.window_size-1)//5))

	def __len__(self):
		return self.length

	def __iter__(self):
		for p in self.prices:
			for k in np.arange(1, 5):
				for offset in range(0, (len(p) - self.window_size-1)//k):
					out = torch.zeros(self.window_size)
					for i in range(self.window_size):
						out[i] = np.sum(p[offset+i*k:offset+(i+1)*k])
					out = out.unsqueeze(1)
					label = np.sum(p[offset+self.window_size*k:offset+(self.window_size+1)*k])
					yield out.squeeze(dim=1), label

In [3]:
# class Transformer_Model(nn.Module):
#     def __init__(self, d_model, in_dim, hidden_dim, out_dim):
#         super().__init__()
#         self.trans = nn.Transformer(d_model=d_model)
#         # self.fc1 = nn.Linear() 
#     def forward(self, x):
#         x = self.trans(x)
#         return x
class FFN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, out_dim)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [4]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"using device: {device}")
epochs = 5
learning_rate = 1e-4
batch_size = 20
window_size = 80
# start = 0
# end = 4

train_dataset = Price_Dataset('Kline/filter', window_size=window_size)
train_loader = du.DataLoader(dataset = train_dataset, batch_size = batch_size)

# net = Transformer_Model(window_size, 1, 1, 1)
net = FFN(80, 40, 1)
optimizer = optim.Adam(net.parameters(), lr = learning_rate)

loss_func = nn.CrossEntropyLoss()
a,b = next(iter(train_loader))
net = net.to(device)
net.train()
# net = 3

using device: cuda:0


FFN(
  (fc1): Linear(in_features=80, out_features=40, bias=True)
  (fc2): Linear(in_features=40, out_features=40, bias=True)
  (fc3): Linear(in_features=40, out_features=1, bias=True)
)

In [5]:
for epoch in range(1 , epochs + 1):
    sum_loss = 0
    for batch_idx, (price, label) in enumerate(tqdm(train_loader)):
        price, label = price.to(device), label.to(device)
        print(label)
        optimizer.zero_grad()
        pred = net(price)
        loss = loss_func(pred.type(torch.float), label.type(torch.long))
        loss.backward()
        optimizer.step()
        sum_loss += loss.item()
    print(f'Epoch {epoch}: Train loss: {sum_loss/batch_idx/batch_size:.8f}')

  0%|          | 0/839 [00:00<?, ?it/s]

tensor([-0.0050,  0.0005,  0.0047, -0.0056, -0.0070,  0.0058,  0.0227, -0.0039,
        -0.0019, -0.0067, -0.0070, -0.0382, -0.0031,  0.0040,  0.0022, -0.0250,
         0.0206, -0.0212, -0.0084,  0.0129], device='cuda:0',
       dtype=torch.float64)


  1%|          | 5/839 [00:02<04:32,  3.06it/s]

tensor([-0.0090,  0.0088, -0.0098,  0.0098, -0.0036,  0.0076, -0.0031,  0.0077,
        -0.0061, -0.0083,  0.0051,  0.0050,  0.0007,  0.0117, -0.0063,  0.0241,
        -0.0047, -0.0226, -0.0102,  0.0062], device='cuda:0',
       dtype=torch.float64)
tensor([ 0.0090,  0.0160,  0.0076, -0.0036,  0.0323, -0.0158,  0.0076, -0.0150,
        -0.0201, -0.0014,  0.0039, -0.0012, -0.0061,  0.0186, -0.0054, -0.0055,
         0.0025, -0.0027, -0.0036, -0.0136], device='cuda:0',
       dtype=torch.float64)
tensor([ 1.1260e-02,  1.1970e-02, -1.0970e-02, -6.5600e-03,  5.5300e-03,
        -2.2000e-03, -5.0500e-03,  9.4400e-03,  9.2100e-03,  5.8200e-03,
        -4.2500e-03,  2.1800e-03,  5.1300e-03, -8.0200e-03, -1.0800e-03,
         1.1230e-02,  5.0000e-05,  1.2130e-02,  6.6000e-04,  3.4500e-03],
       device='cuda:0', dtype=torch.float64)
tensor([ 0.0043, -0.0141, -0.0047, -0.0063, -0.0053,  0.0014, -0.0037,  0.0057,
        -0.0066, -0.0216,  0.0084,  0.0117, -0.0011,  0.0066, -0.0016, -0.0007,
  

  1%|▏         | 11/839 [00:02<01:46,  7.79it/s]

...

tensor([ 0.0062,  0.0028, -0.0407, -0.0241, -0.0975, -0.0287, -0.1487, -0.0647,
        -0.0224, -0.0480,  0.0078, -0.0165, -0.0195, -0.0710, -0.0978, -0.0337,
        -0.0346,  0.1011,  0.0242, -0.0851], device='cuda:0',
       dtype=torch.float64)
tensor([-0.1835, -0.2242, -0.1572, -0.1925, -0.1949, -0.1180, -0.1374,  0.0405,
         0.2885,  0.1648,  0.1283,  0.1100, -0.1040, -0.1158,  0.0013, -0.0730,
         0.0043,  0.0331, -0.0056, -0.0193], device='cuda:0',
       dtype=torch.float64)
tensor([ 0.0543,  0.0666,  0.0456,  0.0538,  0.0208, -0.0104,  0.0150,  0.0011,
        -0.0478, -0.0101, -0.0210,  0.0283,  0.0480,  0.0298,  0.0746,  0.0629,
         0.0805,  0.0907,  0.0890,  0.0107], device='cuda:0',
       dtype=torch.float64)
tensor([-0.0089, -0.0546, -0.1017, -0.0196, -0.0177,  0.0191,  0.0574,  0.0525,
         0.0368,  0.0189, -0.0078, -0.0234, -0.0334,  0.0019,  0.0080, -0.0004,
        -0.0051,  0.0068, -0.0026,  0.0094], device='cuda:0',
       dtype=torch.float64)


 11%|█▏        | 95/839 [00:04<00:20, 36.67it/s]

tensor([-0.0086,  0.0079,  0.0087,  0.0057,  0.0419,  0.0937,  0.0612,  0.0514,
         0.0228, -0.0550,  0.0030,  0.0121,  0.0195,  0.0158, -0.0228, -0.0242,
         0.0023,  0.0718,  0.0596,  0.0584], device='cuda:0',
       dtype=torch.float64)
tensor([ 0.0009, -0.0309, -0.0178, -0.0134, -0.0178, -0.0418, -0.1523, -0.1254,
        -0.0852, -0.0725,  0.0789,  0.0285,  0.0310,  0.0348, -0.0047,  0.0157,
        -0.0174, -0.0265, -0.0032, -0.0126], device='cuda:0',
       dtype=torch.float64)
tensor([ 0.0139,  0.0161,  0.0034, -0.0182, -0.0272, -0.0129, -0.0207,  0.0052,
         0.0139, -0.0083, -0.0245, -0.0081, -0.0360, -0.0009,  0.0438,  0.0242,
         0.0481,  0.0203,  0.0029, -0.0056], device='cuda:0',
       dtype=torch.float64)
tensor([-6.5300e-03,  2.0000e-04, -6.2400e-03, -2.3800e-03,  2.9000e-03,
         7.6300e-03, -1.5200e-03, -3.1290e-02, -2.2980e-02, -2.7980e-02,
        -4.2200e-03,  3.5600e-02, -1.3890e-02,  2.0000e-01, -1.0400e-01,
        -5.6000e-02, -1.0630e+0

 11%|█▏        | 95/839 [00:05<00:40, 18.24it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.